# Table of Content

<a name="outline"></a>

## Setup

- [A](#seca) External Imports
- [B](#secb) Internal Imports
- [C](#secc) Configurations and Paths 
- [D](#secd) Patient Interface and Train/Val/Test Partitioning
- [E](#sece) General Utility Functions


## Training

- [1](#sec1) Training ICE-NODE and The Baselines on MIMIC-III
- [2](#sec2) Training ICE-NODE and The Baselines on MIMIC-IV

<a name="seca"></a>

### A External Imports [^](#outline)

In [1]:
import sys
import os
import glob
import random
from collections import defaultdict
from pathlib import Path

from IPython.display import display

import pandas as pd

from tqdm import tqdm
import jax
jax.config.update('jax_platform_name', 'cpu')

<a name="secb"></a>

### B Internal Imports [^](#outline)

In [2]:
%load_ext autoreload
%autoreload 2

import train as T
import common as C


/home/asem/GP/env/icenode-dev/lib/python3.8/site-packages/haiku/_src/data_structures.py:37: FutureWarning: jax.tree_structure is deprecated, and will be removed in a future release. Use jax.tree_util.tree_structure instead.
  PyTreeDef = type(jax.tree_structure(None))


In [3]:
# HOME and DATA_STORE are arbitrary, change as appropriate.
HOME = os.environ.get('HOME')
DATA_STORE = f'{HOME}/GP/ehr-data'
DATA_FILE = os.path.join(DATA_STORE, 'cprd-data/DUMMY_DATA.csv')
SOURCE_DIR = os.path.abspath("..")

<a name="secd"></a>

### D Configurations and Paths [^](#outline)

**Assign** MIMIC-III and MIMIC-IV directory paths into `mimic3` and `mimic4` variables.

In [4]:
output_dir = 'cprd_artefacts'
Path(output_dir).mkdir(parents=True, exist_ok=True)

In [5]:
with C.modified_environ(DATA_FILE=DATA_FILE):
    cprd_dataset = C.datasets['CPRD']
   

In [6]:
"""
optimal hyperparams re: each model.
"""

model_config = {
    'ICE-NODE': f'{SOURCE_DIR}/expt_configs/cprd/icenode_2lr.json' ,
    'ICE-NODE_UNIFORM': f'{SOURCE_DIR}/expt_configs/cprd/icenode_2lr.json' ,
    'GRU': f'{SOURCE_DIR}/expt_configs/cprd/gru.json' ,
    'RETAIN': f'{SOURCE_DIR}/expt_configs/cprd/retain.json',
    'LogReg': f'{SOURCE_DIR}/expt_configs/cprd/window_logreg.json'
}

model_config = {clf: C.load_config(file) for clf, file in model_config.items()}

clfs = ['ICE-NODE', 'ICE-NODE_UNIFORM', 'GRU', 'RETAIN', 'LogReg']

In [7]:
cprd_train_output_dir = {clf: f'{output_dir}/train/{clf}' for clf in clfs}

[Path(d).mkdir(parents=True, exist_ok=True) for d in cprd_train_output_dir.values()]

[None, None, None, None, None]

In [8]:
cprd_reporters = T.make_reporters(clfs, cprd_train_output_dir)

<a name="sece"></a>

### E Patient Interface and Train/Val/Test Patitioning [^](#outline)

In [9]:
code_scheme = {
    'dx': 'dx_cprd_ltc9809',
    'dx_outcome': 'dx_cprd_ltc9809'
}
cprd_interface = C.Subject_JAX.from_dataset(cprd_dataset, code_scheme=code_scheme)
cprd_splits = cprd_interface.random_splits(split1=0.7, split2=0.85, random_seed=42)


In [10]:
cprd_percentiles = cprd_interface.dx_outcome_by_percentiles(20)
cprd_train_percentiles = cprd_interface.dx_outcome_by_percentiles(20, cprd_splits[0])

In [11]:
cprd_models = T.init_models(clfs, model_config, cprd_interface, cprd_splits[0])


/home/asem/GP/env/icenode-dev/lib/python3.8/site-packages/haiku/_src/data_structures.py:144: FutureWarning: jax.tree_flatten is deprecated, and will be removed in a future release. Use jax.tree_util.tree_flatten instead.
  leaves, treedef = jax.tree_flatten(tree)
/home/asem/GP/env/icenode-dev/lib/python3.8/site-packages/haiku/_src/data_structures.py:145: FutureWarning: jax.tree_unflatten is deprecated, and will be removed in a future release. Use jax.tree_util.tree_unflatten instead.
  return jax.tree_unflatten(treedef, leaves)


<a name="sec1"></a>

### 1 Training ICE-NODE and The Baselines on MIMIC-III [^](#outline)

#### ICE-NODE

In [12]:
## TODO: This may take a long time, a pretrained model already exists in (yy).
cprd_trained_icenode = T.train(cprd_models['ICE-NODE'], config=model_config['ICE-NODE'], 
                             splits=cprd_splits, code_groups=cprd_train_percentiles,
                             reporters=cprd_reporters['ICE-NODE'])

  0%|                                                                                                                                  | 0/60 [00:00<?, ?it/s]/home/asem/GP/env/icenode-dev/lib/python3.8/site-packages/haiku/_src/data_structures.py:144: FutureWarning: jax.tree_flatten is deprecated, and will be removed in a future release. Use jax.tree_util.tree_flatten instead.
  leaves, treedef = jax.tree_flatten(tree)
/home/asem/GP/env/icenode-dev/lib/python3.8/site-packages/haiku/_src/data_structures.py:145: FutureWarning: jax.tree_unflatten is deprecated, and will be removed in a future release. Use jax.tree_util.tree_unflatten instead.
  return jax.tree_unflatten(treedef, leaves)
/home/asem/GP/env/icenode-dev/lib/python3.8/site-packages/haiku/_src/data_structures.py:144: FutureWarning: jax.tree_flatten is deprecated, and will be removed in a future release. Use jax.tree_util.tree_flatten instead.
  leaves, treedef = jax.tree_flatten(tree)
/home/asem/GP/env/icenode-dev/lib/python3.8/

 35%|██████████████████████████████████████████▎                                                                              | 21/60 [01:23<01:10,  1.81s/it]/home/asem/GP/ICE-NODE/notebooks/../icenode/metric/stat.py:215: RuntimeWarning: invalid value encountered in long_scalars
  rate[k][f'ACC-P{i}-k{k}'] = group_true_positive.sum(
 37%|████████████████████████████████████████████▎                                                                            | 22/60 [01:25<01:08,  1.82s/it]/home/asem/GP/ICE-NODE/notebooks/../icenode/metric/stat.py:215: RuntimeWarning: invalid value encountered in long_scalars
  rate[k][f'ACC-P{i}-k{k}'] = group_true_positive.sum(
 38%|██████████████████████████████████████████████▍                                                                          | 23/60 [01:27<01:07,  1.83s/it]/home/asem/GP/ICE-NODE/notebooks/../icenode/metric/stat.py:215: RuntimeWarning: invalid value encountered in long_scalars
  rate[k][f'ACC-P{i}-k{k}'] = group_true_positive.

 75%|██████████████████████████████████████████████████████████████████████████████████████████▊                              | 45/60 [02:07<00:26,  1.75s/it]/home/asem/GP/ICE-NODE/notebooks/../icenode/metric/stat.py:215: RuntimeWarning: invalid value encountered in long_scalars
  rate[k][f'ACC-P{i}-k{k}'] = group_true_positive.sum(
 77%|████████████████████████████████████████████████████████████████████████████████████████████▊                            | 46/60 [02:08<00:24,  1.75s/it]/home/asem/GP/ICE-NODE/notebooks/../icenode/metric/stat.py:215: RuntimeWarning: invalid value encountered in long_scalars
  rate[k][f'ACC-P{i}-k{k}'] = group_true_positive.sum(
 78%|██████████████████████████████████████████████████████████████████████████████████████████████▊                          | 47/60 [02:10<00:22,  1.76s/it]/home/asem/GP/ICE-NODE/notebooks/../icenode/metric/stat.py:215: RuntimeWarning: invalid value encountered in long_scalars
  rate[k][f'ACC-P{i}-k{k}'] = group_true_positive.

#### ICE-NODE_UNIFORM

In [13]:
## TODO: This can take up to (xx), trained model already exist in (yy).
cprd_trained_icenode_uni = T.train(cprd_models['ICE-NODE_UNIFORM'], config=model_config['ICE-NODE_UNIFORM'], 
                                 splits=cprd_splits, code_groups=cprd_train_percentiles,
                                 reporters=cprd_reporters['ICE-NODE_UNIFORM'])


  0%|                                                                                                                                  | 0/60 [00:00<?, ?it/s]/home/asem/GP/env/icenode-dev/lib/python3.8/site-packages/haiku/_src/data_structures.py:144: FutureWarning: jax.tree_flatten is deprecated, and will be removed in a future release. Use jax.tree_util.tree_flatten instead.
  leaves, treedef = jax.tree_flatten(tree)
/home/asem/GP/env/icenode-dev/lib/python3.8/site-packages/haiku/_src/data_structures.py:145: FutureWarning: jax.tree_unflatten is deprecated, and will be removed in a future release. Use jax.tree_util.tree_unflatten instead.
  return jax.tree_unflatten(treedef, leaves)
/home/asem/GP/env/icenode-dev/lib/python3.8/site-packages/haiku/_src/data_structures.py:144: FutureWarning: jax.tree_flatten is deprecated, and will be removed in a future release. Use jax.tree_util.tree_flatten instead.
  leaves, treedef = jax.tree_flatten(tree)
/home/asem/GP/env/icenode-dev/lib/python3.8/

 35%|██████████████████████████████████████████▎                                                                              | 21/60 [01:01<00:40,  1.03s/it]/home/asem/GP/ICE-NODE/notebooks/../icenode/metric/stat.py:215: RuntimeWarning: invalid value encountered in long_scalars
  rate[k][f'ACC-P{i}-k{k}'] = group_true_positive.sum(
 37%|████████████████████████████████████████████▎                                                                            | 22/60 [01:02<00:38,  1.01s/it]/home/asem/GP/ICE-NODE/notebooks/../icenode/metric/stat.py:215: RuntimeWarning: invalid value encountered in long_scalars
  rate[k][f'ACC-P{i}-k{k}'] = group_true_positive.sum(
 38%|██████████████████████████████████████████████▍                                                                          | 23/60 [01:03<00:37,  1.02s/it]/home/asem/GP/ICE-NODE/notebooks/../icenode/metric/stat.py:215: RuntimeWarning: invalid value encountered in long_scalars
  rate[k][f'ACC-P{i}-k{k}'] = group_true_positive.

 75%|██████████████████████████████████████████████████████████████████████████████████████████▊                              | 45/60 [01:25<00:14,  1.03it/s]/home/asem/GP/ICE-NODE/notebooks/../icenode/metric/stat.py:215: RuntimeWarning: invalid value encountered in long_scalars
  rate[k][f'ACC-P{i}-k{k}'] = group_true_positive.sum(
 77%|████████████████████████████████████████████████████████████████████████████████████████████▊                            | 46/60 [01:26<00:13,  1.01it/s]/home/asem/GP/ICE-NODE/notebooks/../icenode/metric/stat.py:215: RuntimeWarning: invalid value encountered in long_scalars
  rate[k][f'ACC-P{i}-k{k}'] = group_true_positive.sum(
 78%|██████████████████████████████████████████████████████████████████████████████████████████████▊                          | 47/60 [01:27<00:12,  1.02it/s]/home/asem/GP/ICE-NODE/notebooks/../icenode/metric/stat.py:215: RuntimeWarning: invalid value encountered in long_scalars
  rate[k][f'ACC-P{i}-k{k}'] = group_true_positive.

#### GRU

In [14]:
## TODO: This can take up to (xx), trained model already exist in (yy).
cprd_trained_gru = T.train(cprd_models['GRU'], config=model_config['GRU'], 
                         splits=cprd_splits, code_groups=cprd_train_percentiles,
                         reporters=cprd_reporters['GRU'])

  0%|                                                                                                                                  | 0/60 [00:00<?, ?it/s]/home/asem/GP/env/icenode-dev/lib/python3.8/site-packages/haiku/_src/data_structures.py:144: FutureWarning: jax.tree_flatten is deprecated, and will be removed in a future release. Use jax.tree_util.tree_flatten instead.
  leaves, treedef = jax.tree_flatten(tree)
/home/asem/GP/env/icenode-dev/lib/python3.8/site-packages/haiku/_src/data_structures.py:145: FutureWarning: jax.tree_unflatten is deprecated, and will be removed in a future release. Use jax.tree_util.tree_unflatten instead.
  return jax.tree_unflatten(treedef, leaves)
/home/asem/GP/env/icenode-dev/lib/python3.8/site-packages/haiku/_src/data_structures.py:144: FutureWarning: jax.tree_flatten is deprecated, and will be removed in a future release. Use jax.tree_util.tree_flatten instead.
  leaves, treedef = jax.tree_flatten(tree)
/home/asem/GP/env/icenode-dev/lib/python3.8/

 35%|██████████████████████████████████████████▎                                                                              | 21/60 [00:11<00:13,  2.90it/s]/home/asem/GP/ICE-NODE/notebooks/../icenode/metric/stat.py:215: RuntimeWarning: invalid value encountered in long_scalars
  rate[k][f'ACC-P{i}-k{k}'] = group_true_positive.sum(
 37%|████████████████████████████████████████████▎                                                                            | 22/60 [00:12<00:12,  2.92it/s]/home/asem/GP/ICE-NODE/notebooks/../icenode/metric/stat.py:215: RuntimeWarning: invalid value encountered in long_scalars
  rate[k][f'ACC-P{i}-k{k}'] = group_true_positive.sum(
 38%|██████████████████████████████████████████████▍                                                                          | 23/60 [00:12<00:12,  2.91it/s]/home/asem/GP/ICE-NODE/notebooks/../icenode/metric/stat.py:215: RuntimeWarning: invalid value encountered in long_scalars
  rate[k][f'ACC-P{i}-k{k}'] = group_true_positive.

 75%|██████████████████████████████████████████████████████████████████████████████████████████▊                              | 45/60 [00:20<00:05,  2.92it/s]/home/asem/GP/ICE-NODE/notebooks/../icenode/metric/stat.py:215: RuntimeWarning: invalid value encountered in long_scalars
  rate[k][f'ACC-P{i}-k{k}'] = group_true_positive.sum(
 77%|████████████████████████████████████████████████████████████████████████████████████████████▊                            | 46/60 [00:20<00:04,  2.86it/s]/home/asem/GP/ICE-NODE/notebooks/../icenode/metric/stat.py:215: RuntimeWarning: invalid value encountered in long_scalars
  rate[k][f'ACC-P{i}-k{k}'] = group_true_positive.sum(
 78%|██████████████████████████████████████████████████████████████████████████████████████████████▊                          | 47/60 [00:20<00:04,  2.92it/s]/home/asem/GP/ICE-NODE/notebooks/../icenode/metric/stat.py:215: RuntimeWarning: invalid value encountered in long_scalars
  rate[k][f'ACC-P{i}-k{k}'] = group_true_positive.

#### RETAIN

In [15]:
## TODO: This can take up to (xx), trained model already exist in (yy).
cprd_trained_retain = T.train(cprd_models['RETAIN'], config=model_config['RETAIN'], 
                         splits=cprd_splits, code_groups=cprd_train_percentiles,
                         reporters=cprd_reporters['RETAIN'])

  0%|                                                                                                                                  | 0/60 [00:00<?, ?it/s]/home/asem/GP/env/icenode-dev/lib/python3.8/site-packages/haiku/_src/data_structures.py:144: FutureWarning: jax.tree_flatten is deprecated, and will be removed in a future release. Use jax.tree_util.tree_flatten instead.
  leaves, treedef = jax.tree_flatten(tree)
/home/asem/GP/env/icenode-dev/lib/python3.8/site-packages/haiku/_src/data_structures.py:145: FutureWarning: jax.tree_unflatten is deprecated, and will be removed in a future release. Use jax.tree_util.tree_unflatten instead.
  return jax.tree_unflatten(treedef, leaves)
/home/asem/GP/env/icenode-dev/lib/python3.8/site-packages/haiku/_src/data_structures.py:144: FutureWarning: jax.tree_flatten is deprecated, and will be removed in a future release. Use jax.tree_util.tree_flatten instead.
  leaves, treedef = jax.tree_flatten(tree)
/home/asem/GP/env/icenode-dev/lib/python3.8/

 35%|██████████████████████████████████████████▎                                                                              | 21/60 [00:21<00:22,  1.70it/s]/home/asem/GP/ICE-NODE/notebooks/../icenode/metric/stat.py:215: RuntimeWarning: invalid value encountered in long_scalars
  rate[k][f'ACC-P{i}-k{k}'] = group_true_positive.sum(
 37%|████████████████████████████████████████████▎                                                                            | 22/60 [00:21<00:22,  1.72it/s]/home/asem/GP/ICE-NODE/notebooks/../icenode/metric/stat.py:215: RuntimeWarning: invalid value encountered in long_scalars
  rate[k][f'ACC-P{i}-k{k}'] = group_true_positive.sum(
 38%|██████████████████████████████████████████████▍                                                                          | 23/60 [00:22<00:20,  1.80it/s]/home/asem/GP/ICE-NODE/notebooks/../icenode/metric/stat.py:215: RuntimeWarning: invalid value encountered in long_scalars
  rate[k][f'ACC-P{i}-k{k}'] = group_true_positive.

 75%|██████████████████████████████████████████████████████████████████████████████████████████▊                              | 45/60 [00:34<00:07,  1.99it/s]/home/asem/GP/ICE-NODE/notebooks/../icenode/metric/stat.py:215: RuntimeWarning: invalid value encountered in long_scalars
  rate[k][f'ACC-P{i}-k{k}'] = group_true_positive.sum(
 77%|████████████████████████████████████████████████████████████████████████████████████████████▊                            | 46/60 [00:34<00:06,  2.02it/s]/home/asem/GP/ICE-NODE/notebooks/../icenode/metric/stat.py:215: RuntimeWarning: invalid value encountered in long_scalars
  rate[k][f'ACC-P{i}-k{k}'] = group_true_positive.sum(
 78%|██████████████████████████████████████████████████████████████████████████████████████████████▊                          | 47/60 [00:34<00:06,  2.04it/s]/home/asem/GP/ICE-NODE/notebooks/../icenode/metric/stat.py:215: RuntimeWarning: invalid value encountered in long_scalars
  rate[k][f'ACC-P{i}-k{k}'] = group_true_positive.

In [16]:
cprd_trained_logreg = T.train(cprd_models['LogReg'], config=model_config['LogReg'], 
                         splits=cprd_splits, code_groups=cprd_train_percentiles,
                         reporters=cprd_reporters['LogReg'])

/home/asem/GP/ICE-NODE/notebooks/../icenode/metric/stat.py:215: RuntimeWarning: invalid value encountered in long_scalars
  rate[k][f'ACC-P{i}-k{k}'] = group_true_positive.sum(
/home/asem/GP/ICE-NODE/notebooks/../icenode/metric/stat.py:215: RuntimeWarning: invalid value encountered in long_scalars
  rate[k][f'ACC-P{i}-k{k}'] = group_true_positive.sum(
